In [53]:
import boto3
import os
import json
import configparser
from botocore.exceptions import ClientError
import psycopg2
from pyspark.sql.functions import concat, col, lit

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
AWS_KEY='KEY'
AWS_SECRET='SECRET'
DWH_CLUSTER_TYPE='multi-node'
DWH_NUM_NODES=4
DWH_NODE_TYPE='dc2.large'

DWH_IAM_ROLE_NAME='RoleForRedshift_FinalProject'
DWH_IAM_ROLE_WITH_ARN='arn:aws:iam::410589112277:role/RoleForRedshift_FinalProject'
DWH_CLUSTER_IDENTIFIER='covid-19-analysis'
DWH_DB='database'
DWH_DB_USER='user'
DWH_DB_PASSWORD='password'
DWH_PORT=5439

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
iam = boto3.client('iam',aws_access_key_id=AWS_KEY,
                     aws_secret_access_key=AWS_SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=AWS_KEY,
                       aws_secret_access_key=AWS_SECRET
                       )
s3 = boto3.resource('s3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Data about Covid-19 cases to be processed from Germany and Italy
#### This s3 bucket should be updated with new cases using API or manual inputs

##### source data #1 - Germany data about COVID-19

In [56]:
germany_data = 's3://udacity-de-capstone-project/germany/covid_de.csv'
germany = spark.read.format('csv').option("header", "true").load(germany_data)
germany = germany.select('state',
                         'county',
                         'age_group',
                         'gender',
                         concat(col('date'),lit('T00:00:00')).alias('dt'),
                         'cases',
                         'deaths',
                         'recovered')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
s3 = boto3.resource('s3')
objects_to_delete = s3.meta.client.list_objects(Bucket="udacity-de-capstone-project", Prefix="germany/staging_table/")

delete_keys = {'Objects' : []}
delete_keys['Objects'] = [{'Key' : k} for k in [obj['Key'] for obj in objects_to_delete.get('Contents', [])]]

s3.meta.client.delete_objects(Bucket="udacity-de-capstone-project", Delete=delete_keys)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '8V8G8T1H0P4H5W0P', 'HostId': 'LCXuUjIjzcJ/C4uDBekafx29Br2a2i0OAoYlmLiW5ZoYBosBPVgF9Qw/wzytWFk5xsmHZOjbsJY=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'LCXuUjIjzcJ/C4uDBekafx29Br2a2i0OAoYlmLiW5ZoYBosBPVgF9Qw/wzytWFk5xsmHZOjbsJY=', 'x-amz-request-id': '8V8G8T1H0P4H5W0P', 'date': 'Sun, 17 Jan 2021 03:40:46 GMT', 'connection': 'close', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Deleted': [{'Key': 'germany/staging_table/part-00001-99bf4578-7f00-4a87-8495-f4744efedb08-c000.json'}, {'Key': 'germany/staging_table/_SUCCESS'}, {'Key': 'germany/staging_table/part-00000-99bf4578-7f00-4a87-8495-f4744efedb08-c000.json'}]}

In [58]:
germany_staging_table_s3_path = 's3://udacity-de-capstone-project/germany/staging_table'
germany.write.format("json").save(germany_staging_table_s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### source data #2 - Italy Cities/Provinces

In [59]:
italy_province_data = 's3://udacity-de-capstone-project/italy/dati-json/dpc-covid19-ita-province.json'
italy_province = spark.read.json(italy_province_data,multiLine=True).withColumnRenamed('data','dt')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
s3 = boto3.resource('s3')
objects_to_delete = s3.meta.client.list_objects(Bucket="udacity-de-capstone-project", Prefix="italy/staging_table/province")

delete_keys = {'Objects' : []}
delete_keys['Objects'] = [{'Key' : k} for k in [obj['Key'] for obj in objects_to_delete.get('Contents', [])]]

s3.meta.client.delete_objects(Bucket="udacity-de-capstone-project", Delete=delete_keys)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '58CAD07819B82CBD', 'HostId': '4bvz4utN1nwE0tzYdYCVyy4jY5s4zSZ2Nhbad4s0FPkDnlxF9DXz20M7IuWwEv3r0YWP1zXCr4s=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '4bvz4utN1nwE0tzYdYCVyy4jY5s4zSZ2Nhbad4s0FPkDnlxF9DXz20M7IuWwEv3r0YWP1zXCr4s=', 'x-amz-request-id': '58CAD07819B82CBD', 'date': 'Sun, 17 Jan 2021 03:40:55 GMT', 'connection': 'close', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Deleted': [{'Key': 'italy/staging_table/province/part-00000-a95f57d7-def5-43e8-9f61-f539811b9503-c000.json'}, {'Key': 'italy/staging_table/province/_SUCCESS'}]}

In [61]:
italy_province_staging_table_s3_path = 's3://udacity-de-capstone-project/italy/staging_table/province'
italy_province.write.format("json").save(italy_province_staging_table_s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### source data #3 - Italy Regions and COVID-19 data

In [62]:
italy_regioni_data = 's3://udacity-de-capstone-project/italy/dati-json/dpc-covid19-ita-regioni.json'
italy_regioni = spark.read.json(italy_regioni_data,multiLine=True).withColumnRenamed('data','dt')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
s3 = boto3.resource('s3')
objects_to_delete = s3.meta.client.list_objects(Bucket="udacity-de-capstone-project", Prefix="italy/staging_table/regioni")

delete_keys = {'Objects' : []}
delete_keys['Objects'] = [{'Key' : k} for k in [obj['Key'] for obj in objects_to_delete.get('Contents', [])]]

s3.meta.client.delete_objects(Bucket="udacity-de-capstone-project", Delete=delete_keys)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'D7616FF53D5D6ACF', 'HostId': '22Dx1VnKX3VkO3d/MXPLaFqhTDp1MiO1ytd2GdkyWy8CmIOEmwC2/b7diWJhUVNbonWnMOXgLvI=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '22Dx1VnKX3VkO3d/MXPLaFqhTDp1MiO1ytd2GdkyWy8CmIOEmwC2/b7diWJhUVNbonWnMOXgLvI=', 'x-amz-request-id': 'D7616FF53D5D6ACF', 'date': 'Sun, 17 Jan 2021 03:41:00 GMT', 'connection': 'close', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Deleted': [{'Key': 'italy/staging_table/regioni/_SUCCESS'}, {'Key': 'italy/staging_table/regioni/part-00000-8bbfa15a-a50b-42a5-aaf8-e791ec61556f-c000.json'}]}

In [64]:
italy_regioni_staging_table_s3_path = 's3://udacity-de-capstone-project/italy/staging_table/regioni'
italy_regioni.write.format("json").save(italy_regioni_staging_table_s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Parameters to create AWS Redshift cluster
#### If IAM Role you plan to use already exists, just set this name in specific parameter and it's going to be used instead of recreate.

In [65]:
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name RoleForRedshift_FinalProject already exists.
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::410589112277:role/RoleForRedshift_FinalProject

In [66]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists

In [67]:
all_keys = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
keys_to_show = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId','IamRoles']

for key in keys_to_show:
  print(f"{key}-> {all_keys[key]}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ClusterIdentifier-> covid-19-analysis
NodeType-> dc2.large
ClusterStatus-> available
MasterUsername-> singaretti
DBName-> database
Endpoint-> {'Address': 'covid-19-analysis.coi6foj1tbge.us-west-2.redshift.amazonaws.com', 'Port': 5439}
NumberOfNodes-> 4
VpcId-> vpc-0e5c237dda5620d8a
IamRoles-> [{'IamRoleArn': 'arn:aws:iam::410589112277:role/RoleForRedshift_FinalProject', 'ApplyStatus': 'in-sync'}]

In [68]:
conn = psycopg2.connect(host='covid-19-analysis.coi6foj1tbge.us-west-2.redshift.amazonaws.com',
                        dbname='database',
                        user='singaretti',
                        password='Singaretti1',
                        port='5439')
conn.set_session(autocommit=True)
cur = conn.cursor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
# DROP TABLES
staging_italy_province_table_drop = 'DROP TABLE IF EXISTS staging_italy_province;'
staging_italy_regioni_table_drop = 'DROP TABLE IF EXISTS staging_italy_regioni;'
staging_germany_table_drop = 'DROP TABLE IF EXISTS staging_germany;'

# CREATE TABLES
staging_italy_province_table_create= ("""
CREATE TABLE IF NOT EXISTS staging_italy_province
(
codice_provincia INT,
codice_regione INT,
dt VARCHAR,
denominazione_provincia VARCHAR,
denominazione_regione VARCHAR,
lat FLOAT,
long FLOAT,
sigla_provincia VARCHAR,
stato VARCHAR,
totale_casi VARCHAR)
""")

staging_italy_regioni_table_create= ("""
CREATE TABLE IF NOT EXISTS staging_italy_regioni
(
codice_regione INT,
dt VARCHAR,
deceduti INT,
denominazione_regione VARCHAR,
dimessi_guariti INT,
lat FLOAT,
long FLOAT,
nuovi_positivi INT,
ricoverati_con_sintomi INT,
stato VARCHAR,
tamponi INT,
terapia_intensiva INT,
totale_casi INT,
totale_ospedalizzati INT,
totale_positivi INT,
variazione_totale_positivi INT)
""")

staging_germany_table_create = ("""
CREATE TABLE IF NOT EXISTS staging_germany
(
state VARCHAR,
county VARCHAR,
age_group VARCHAR,
gender VARCHAR,
dt VARCHAR,
cases INT,
deaths INT,
recovered INT)
""")

# COPYING TO STAGING TABLES
staging_italy_province_copy = ("""COPY staging_italy_province FROM '{}' iam_role '{}' FORMAT AS JSON 'auto'""").format(italy_province_staging_table_s3_path, DWH_IAM_ROLE_WITH_ARN)

staging_italy_regioni_copy = ("""COPY staging_italy_regioni FROM '{}' iam_role '{}' FORMAT AS JSON 'auto'""").format(italy_regioni_staging_table_s3_path, DWH_IAM_ROLE_WITH_ARN)

staging_germany_copy = ("""COPY staging_germany FROM '{}' iam_role '{}' FORMAT AS JSON 'auto'""").format(germany_staging_table_s3_path, DWH_IAM_ROLE_WITH_ARN)

# QUERY LISTS
create_table_queries = [staging_italy_province_table_create, staging_italy_regioni_table_create, staging_germany_table_create]
drop_table_queries = [staging_italy_province_table_drop, staging_italy_regioni_table_drop, staging_germany_table_drop]
copy_table_queries = [staging_germany_copy, staging_italy_province_copy, staging_italy_regioni_copy]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
def drop_tables(cur, conn):
    """
    - drop table if exists in AWS Redshift
    """
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

def create_tables(cur, conn):
    """
    - Create table statements for AWS Redshift
    """
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()
        
def load_staging_tables(cur, conn):
    """
    - Load files from s3 bucket into staging tables
    """
    for query in copy_table_queries:
        cur.execute(query)
        conn.commit()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
drop_tables(cur,conn)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
create_tables(cur, conn)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
load_staging_tables(cur, conn)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…